In [89]:
import pandas as pd 
import numpy as np
import requests
import json
from datetime import datetime

### Por medio de un Json File vamos a manejar la las Key de la API y de Redshift y tambien usuario Redshift.

In [249]:
with open("config.json") as config_file:
    config=json.load(config_file)

In [250]:
Api_Key=config["API_KEY"]

In [251]:
Redshift_Key=config["Reshift_KEY"]

In [252]:
Redshift_User=config["Redshift_User"]

### El proposito de esta extraccion es mantener un historico del Clima 6 lugares lugares turisticos de Argentina a lo largo del tiempo

In [213]:
Destinos=["Buenos_Aires","Bariloche","Mendoza","Jujuy","Salta","Mar_del_plata"]

## Se crea una funcion que crea una DataFrame extrayendo algunos datos de cada una de las ciudades turisticas para posteriormente subirlo a Redshift

In [253]:
def weather_df(lista_ciudades):
    
    conn = http.client.HTTPSConnection("api.weatherapi.com")

    headers = {
        'X-RapidAPI-Key': Api_Key,
        'X-RapidAPI-Host': "api.weatherapi.com"
    }

    
    df_final=[]
    
    for x in lista_ciudades:
        code="/v1/current.json?key=34eeba99ba8e404d947211802230909&q="+str(x)+"&aqi=no"
        conn.request("GET", code , headers=headers)
        res = conn.getresponse()
        data = res.read()
        weather_df=json.loads(data)
        date=weather_df.get("location",{}).get("localtime")
        city_name=weather_df.get("location",{}).get("name")
        region=weather_df.get("location",{}).get("region")
        country=weather_df.get("location",{}).get("country")
        weather=weather_df.get("current",{}).get("temp_c")
        humidity=weather_df.get("current",{}).get("humidity")
        precipitation=weather_df.get("current",{}).get("precip_mm")
        last_updated=weather_df.get("current",{}).get("last_updated")
        registro={"Date":date,"City_Name":city_name,"Region":region,"Country":country,"Weather":weather,"Humidity":humidity,
                  "Precipitation":precipitation,"Last_Updated":last_updated}
        df_final.append(registro)
    df_final=pd.DataFrame.from_dict(df_final)
            
    return df_final

In [285]:
weather=weather_df(Destinos)

In [286]:
weather["Date"] = pd.to_datetime(weather["Date"], format='%Y-%m-%d %H:%M').dt.date

In [287]:
weather["Last_Updated"]=pd.to_datetime(weather["Last_Updated"], format='%Y-%m-%d %H:%M')

In [288]:
weather

,Date,City_Name,Region,Country,Weather,Humidity,Precipitation,Last_Updated
0,2023-09-09,Buenos Aires,Distrito Federal,Argentina,13.9,70,0.0,2023-09-09 22:00:00
1,2023-09-09,Bariloche,Rio Negro,Argentina,1.3,90,0.0,2023-09-09 22:00:00
2,2023-09-09,Mendoza,Mendoza,Argentina,14.6,36,0.0,2023-09-09 22:00:00
3,2023-09-09,Jujuy,Jujuy,Argentina,9.3,65,0.0,2023-09-09 22:00:00
4,2023-09-09,Salta,Salta,Argentina,13.1,59,0.0,2023-09-09 22:00:00
5,2023-09-09,Mar Del Plata,Buenos Aires,Argentina,11.5,74,0.0,2023-09-09 21:45:00


## Chequeamos los Missing Values del DataFrame

In [290]:
weather.isnull().sum()

Date             0
City_Name        0
Region           0
Country          0
Weather          0
Humidity         0
Precipitation    0
Last_Updated     0
dtype: int64

## Crear conexion a Redshift

In [291]:
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user=Redshift_User


try:  
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=Redshift_Key,
        port='5439')
    print("Conectado a Redshift con éxito!")

        
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [281]:
# Crear un cursor:
cur = conn.cursor()

# Ejecutar la sentencia DROP TABLE:
cur.execute("DROP TABLE IF EXISTS Weather")

# Hacer commit para aplicar los cambios:
conn.commit()

In [282]:
#Crear la tabla si no existe:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Weather (
            Date TIMESTAMP,
            City_Name VARCHAR(20),
            Region VARCHAR(25),
            Country VARCHAR(25),
            Weather FLOAT,
            Humidity INT,
            Precipitation FLOAT,
            Last_Updated TIMESTAMP
        )
    """)
    conn.commit()

In [292]:
from psycopg2.extras import execute_values

with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO Weather (Date, City_Name, Region, Country, Weather, Humidity, Precipitation, Last_Updated)
        VALUES %s
        ''',
        [tuple(row) for row in weather.values],
        page_size=len(weather)
    )
    conn.commit()